In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json

# General Dataset

## 1 General Johns Hopkins University Dataset

In [2]:
df = pd.read_csv("data/Enigma-JHU.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123687 entries, 0 to 123686
Data columns (total 12 columns):
fips              109275 non-null float64
admin2            109199 non-null object
province_state    114568 non-null object
country_region    123687 non-null object
last_update       123687 non-null object
latitude          123504 non-null float64
longitude         123504 non-null float64
confirmed         123668 non-null float64
deaths            123246 non-null float64
recovered         123299 non-null float64
active            118673 non-null float64
combined_key      123687 non-null object
dtypes: float64(7), object(5)
memory usage: 11.3+ MB


In [4]:
df["last_update"] = df["last_update"].apply(lambda x : datetime.datetime.strptime(x[:10], '%Y-%m-%d'))

In [5]:
df = df.fillna(0)

In [6]:
df.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,0.0,0,Anhui,China,2020-01-22,31.826,117.226,1.0,0.0,0.0,0.0,"Anhui, China"
1,0.0,0,Beijing,China,2020-01-22,40.182,116.414,14.0,0.0,0.0,0.0,"Beijing, China"
2,0.0,0,Chongqing,China,2020-01-22,30.057,107.874,6.0,0.0,0.0,0.0,"Chongqing, China"
3,0.0,0,Fujian,China,2020-01-22,26.079,117.987,1.0,0.0,0.0,0.0,"Fujian, China"
4,0.0,0,Gansu,China,2020-01-22,36.061,103.834,0.0,0.0,0.0,0.0,"Gansu, China"


In [7]:
per_day = pd.DataFrame(df.groupby(['country_region', 'last_update'])['confirmed', 'deaths', 'recovered'].sum())

In [8]:
per_day = per_day.reset_index()

In [9]:
per_day['last_update'] = per_day['last_update'].apply(lambda date: date.isoformat())

In [10]:
per_day.head()

,country_region,last_update,confirmed,deaths,recovered
0,Afghanistan,2020-02-24T00:00:00,1.0,0.0,0.0
1,Afghanistan,2020-03-08T00:00:00,4.0,0.0,0.0
2,Afghanistan,2020-03-10T00:00:00,5.0,0.0,0.0
3,Afghanistan,2020-03-11T00:00:00,14.0,0.0,0.0
4,Afghanistan,2020-03-14T00:00:00,11.0,0.0,0.0


In [11]:
# Selected countries we want to show
per_day_selected_countries = per_day[((((per_day['country_region'] == 'US') |\
                                     (per_day['country_region'] == 'Switzerland')) |\
                                     (per_day['country_region'] == 'Italy')) |\
                                     (per_day['country_region'] == 'France')) |\
                                     (per_day['country_region'] == 'Germany')]

per_day_selected_countries = per_day_selected_countries.rename({'country_region':'topicName', 'last_update':'date'}, axis=1)

# Get unique number for each country
df_map_numberToCountry = pd.DataFrame(per_day_selected_countries['topicName'].unique()).reset_index()
df_map_numberToCountry['index'] += 1
df_map_numberToCountry = df_map_numberToCountry.rename({'index':'name', 0:'topicName'}, axis=1)

# Merge with prior dataset
per_day_selected_countries = per_day_selected_countries.merge(df_map_numberToCountry, on='topicName')

# Divide dataset into 3, one for each plot
per_day_selected_countries_cases = per_day_selected_countries[['topicName', 'name', 'date', 'confirmed']]
per_day_selected_countries_deaths = per_day_selected_countries[['topicName', 'name', 'date', 'deaths']]
per_day_selected_countries_recovered = per_day_selected_countries[['topicName', 'name', 'date', 'recovered']]

# Renaming to correspond to Britecharts data format
per_day_selected_countries_cases = per_day_selected_countries_cases.rename({'confirmed':'value'}, axis=1)
per_day_selected_countries_deaths = per_day_selected_countries_cases.rename({'deaths':'value'}, axis=1)
per_day_selected_countries_recovered = per_day_selected_countries_cases.rename({'recovered':'value'}, axis=1)

# Reformat to correspond to Britecharts data format
per_day_selected_countries_cases_json = per_day_selected_countries_cases.to_dict(orient='records')
per_day_selected_countries_deaths_json = per_day_selected_countries_deaths.to_dict(orient='records')
per_day_selected_countries_recovered_json = per_day_selected_countries_recovered.to_dict(orient='records')

# Saving to json format
json1 = json.dumps(per_day_selected_countries_cases_json)
json2 = json.dumps(per_day_selected_countries_deaths_json)
json3 = json.dumps(per_day_selected_countries_recovered_json)

f1 = open('per_day_selected_countries_cases_json.json', 'w')
f2 = open('per_day_selected_countries_deaths_json.json', 'w')
f3 = open('per_day_selected_countries_recovered_json.json', 'w')

f1.write(json1)
f2.write(json2)
f3.write(json3)

f1.close()
f2.close()
f3.close()


In [12]:
per_day.to_csv('Data_per_day_country.csv', index = False)

In [13]:
for_map = df[["last_update","confirmed","deaths","recovered" , "latitude", "longitude"]]

In [14]:
for_map.head()

,last_update,confirmed,deaths,recovered,latitude,longitude
0,2020-01-22,1.0,0.0,0.0,31.826,117.226
1,2020-01-22,14.0,0.0,0.0,40.182,116.414
2,2020-01-22,6.0,0.0,0.0,30.057,107.874
3,2020-01-22,1.0,0.0,0.0,26.079,117.987
4,2020-01-22,0.0,0.0,0.0,36.061,103.834


In [15]:
for_map.to_csv('for_map.csv', index = False)

## 2 Gender/age Dataset

In [16]:
df1 = pd.read_csv("data/covid19_sexe.csv", sep = ',')

In [17]:
df1.head()

,Country,Sex-disaggregated?,Date,Cases,Cases (% male),Cases (% female),Deaths,deaths (% male),deaths (% female),Deaths among confirmed cases (male),Deaths in confirmed cases (female),Deaths in confirmed cases (Male:female ratio),Sources
0,Dominican Republic,Yes,26.04.20,"6,293",54.0,46.0,282,78.0,22.0,6.5%,2.1%,3.0,Source
1,Thailand,Yes,27.04.20,"2,938",55.0,45.0,52,77.0,23.0,2.5%,0.9%,2.7,Source
2,Greece,Yes,27.04.20,"2,324",55.0,45.0,132,76.0,24.0,7.8%,3.1%,2.5,Source
3,Romania,Yes,26.04.20,"11,313",45.0,55.0,619,64.0,36.0,7.9%,3.5%,2.3,Source
4,The Netherlands,Yes,28.04.20,"38,365",38.0,62.0,"4,566",57.0,43.0,18.1%,8.1%,2.2,Source


In [18]:
df1.dropna(subset = ["deaths (% male)"], inplace = True)

In [19]:
df_cl = df1[["Country","Cases (% male)", "Cases (% female)", "deaths (% male)","deaths (% female)", "Deaths among confirmed cases (male)", "Deaths in confirmed cases (female)", "Deaths in confirmed cases (Male:female ratio)"]]

In [20]:
df_cl.head()

,Country,Cases (% male),Cases (% female),deaths (% male),deaths (% female),Deaths among confirmed cases (male),Deaths in confirmed cases (female),Deaths in confirmed cases (Male:female ratio)
0,Dominican Republic,54.0,46.0,78.0,22.0,6.5%,2.1%,3.0
1,Thailand,55.0,45.0,77.0,23.0,2.5%,0.9%,2.7
2,Greece,55.0,45.0,76.0,24.0,7.8%,3.1%,2.5
3,Romania,45.0,55.0,64.0,36.0,7.9%,3.5%,2.3
4,The Netherlands,38.0,62.0,57.0,43.0,18.1%,8.1%,2.2


In [21]:
df_cl.to_csv("Gender_per_country.csv")

## 3 SARS Dataset

In [22]:
sars = pd.read_csv("data/sars_2003_complete_dataset_clean.csv", sep = ',')

In [23]:
sars.head()

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0


In [24]:
sars[["Country", "Date", "Cumulative number of case(s)", "Number of deaths", "Number recovered"]].to_csv("sars_country_date.csv", index = False)

## 4 Ebola Dataset

In [25]:
ebola = pd.read_csv("data/ebola_2014_2016_clean.csv")

In [26]:
ebola.head()

,Country,Date,No. of suspected cases,No. of probable cases,No. of confirmed cases,"No. of confirmed, probable and suspected cases",No. of suspected deaths,No. of probable deaths,No. of confirmed deaths,"No. of confirmed, probable and suspected deaths"
0,Guinea,2014-08-29,25.0,141.0,482.0,648.0,2.0,141.0,287.0,430.0
1,Nigeria,2014-08-29,3.0,1.0,15.0,19.0,0.0,1.0,6.0,7.0
2,Sierra Leone,2014-08-29,54.0,37.0,935.0,1026.0,8.0,34.0,380.0,422.0
3,Liberia,2014-08-29,382.0,674.0,322.0,1378.0,168.0,301.0,225.0,694.0
4,Sierra Leone,2014-09-05,78.0,37.0,1146.0,1261.0,11.0,37.0,443.0,491.0


In [27]:
ebola.fillna(0, inplace = True)

In [28]:
ebola[["Country", "Date", "No. of confirmed cases", "No. of confirmed deaths"]].to_csv("ebola_country_date.csv", index = False)

# Government Dataset and Testing Dataset

#### Testing Dataset

In [29]:
tests = pd.read_csv('data/covid19_testing.csv')

In [30]:
tests.head()

,Entity,Code,Date,Total tests per thousand
0,Argentina,ARG,"Apr 8, 2020",0.295
1,Argentina,ARG,"Apr 9, 2020",0.329
2,Argentina,ARG,"Apr 10, 2020",0.362
3,Argentina,ARG,"Apr 11, 2020",0.399
4,Argentina,ARG,"Apr 13, 2020",0.437


In [31]:
tests[['Entity', 'Date', 'Total tests per thousand']].isnull().values.any()

False

In [32]:
tests['Code'].isnull().values.any()

True

We can thus delete the `Code` columns from the dataset

In [33]:
tests = tests.drop(columns = ['Code'])

Check if no two dates are the same for a given Country

In [34]:
tests.groupby('Entity')['Date'].apply(lambda x: x.duplicated().any()).unique()

array([False])

In [35]:
selected_countries = ['United States', 'France', 'Belgium', 'Germany']

In [36]:
tests = tests[tests['Entity'].isin(selected_countries)]

In [37]:
tests.head()

,Entity,Date,Total tests per thousand
256,Belgium,"Mar 1, 2020",0.005
257,Belgium,"Mar 2, 2020",0.029
258,Belgium,"Mar 3, 2020",0.070
259,Belgium,"Mar 4, 2020",0.126
260,Belgium,"Mar 5, 2020",0.187


#### Governement Dataset

In [38]:
gov_oxford = pd.read_csv('data/gov_oxford.csv')

In [39]:
gov_oxford.head()

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,LegacyStringencyIndex,LegacyStringencyIndexForDisplay
0,Aruba,ABW,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,Aruba,ABW,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,Aruba,ABW,20200103,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,Aruba,ABW,20200104,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,Aruba,ABW,20200105,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [40]:
gov_oxford = gov_oxford.drop(columns = ['S1_Notes', 'S1_IsGeneral', 'S2_Notes', 'S2_IsGeneral', 'S3_Notes', 'S3_IsGeneral', 'S4_Notes', 'S4_IsGeneral', 'S5_Notes', 'S5_IsGeneral', 'S6_Notes', 'S6_IsGeneral', 'S7_Notes', 'S8_Notes', 'S9_Notes', 'S10_Notes', 'S11_Notes', 'S12_Notes', 'S13_Notes', 'Unnamed: 39'])

KeyError: "['S1_Notes' 'S1_IsGeneral' 'S2_Notes' 'S2_IsGeneral' 'S3_Notes'\n 'S3_IsGeneral' 'S4_Notes' 'S4_IsGeneral' 'S5_Notes' 'S5_IsGeneral'\n 'S6_Notes' 'S6_IsGeneral' 'S7_Notes' 'S8_Notes' 'S9_Notes' 'S10_Notes'\n 'S11_Notes' 'S12_Notes' 'S13_Notes' 'Unnamed: 39'] not found in axis"

We want every country to be compared on the same date range

In [ ]:
len(gov_oxford['CountryName'].unique())

In [ ]:
len(gov_oxford['Date'].unique())

In [ ]:
gov_oxford[['CountryName', 'Date']].isnull().values.any()

In [ ]:
full_dates = gov_oxford[['Date', 'CountryName']].groupby('Date').count()
full_dates = full_dates[full_dates['CountryName'] == len(gov_oxford['CountryName'].unique())]
full_dates.head()

In [ ]:
full_dates = full_dates.reset_index().drop(columns = ['CountryName'])
gov_oxford_map = gov_oxford[gov_oxford['Date'].isin(full_dates)]

In [ ]:
def convert_date_appearance():
    

In [ ]:
gov_map_si = gov_oxford_map[['CountryName', 'Date', 'StringencyIndexForDisplay']]

In [ ]:
gov_map_school = gov_oxford_map[['CountryName', 'Date', 'S1_School closing']]

In [ ]:
gov_map_work = gov_oxford_map[['CountryName', 'Date', 'S2_Workplace closing']]

In [ ]:
gov_map_events = gov_oxford_map[['CountryName', 'Date', 'S3_Cancel public events']]

In [ ]:
gov_map_transport = gov_oxford_map[['CountryName', 'Date', 'S4_Close public transport']]

In [ ]:
gov_map_info = gov_oxford_map[['CountryName', 'Date', 'S5_Public information campaigns']]

In [ ]:
gov_map_movement = gov_oxford_map[['CountryName', 'Date', 'S6_Restrictions on internal movement']]

In [ ]:
gov_map_travel_controls = gov_oxford_map[['CountryName', 'Date', 'S7_International travel controls']]

In [ ]:
gov_map_fiscal = gov_oxford_map[['CountryName', 'Date', 'S8_Fiscal measures']]

In [ ]:
gov_map_monetary = gov_oxford_map[['CountryName', 'Date', 'S9_Monetary measures']]

In [ ]:
gov_map_health_investment = gov_oxford_map[['CountryName', 'Date', 'S10_Emergency investment in health care']]

In [ ]:
gov_map_vaccine_investment = gov_oxford_map[['CountryName', 'Date', 'S11_Investment in Vaccines']]

In [ ]:
gov_map_testing = gov_oxford_map[['CountryName', 'Date', 'S12_Testing framework']]

In [ ]:
gov_map_contact_tracing = gov_oxford_map[['CountryName', 'Date', 'S13_Contact tracing']]

Country selection

In [ ]:
selected_countries = ['United States', 'France', 'Germany', 'Belgium']

In [ ]:
gov_oxford_select = gov_oxford[gov_oxford['CountryName'].isin(selected_countries)]

In [ ]:
gov_oxford_select.head()

In [ ]:
gov_oxford_select[['CountryName', 'Date', 'ConfirmedCases', 'ConfirmedDeaths']].isnull().values.any()

In [ ]:
gov_selected_cases_si = gov_oxford_select[['CountryName', 'Date', 'ConfirmedCases', 'StringencyIndexForDisplay']]

Non economical measures

In [ ]:
non_economical_measures = ['CountryName', 'Date', 'ConfirmedCases', 'S1_School closing', 'S2_Workplace closing', 'S3_Cancel public events', 'S4_Close public transport', 'S5_Public information campaigns', 'S6_Restrictions on internal movement', 'S13_Contact tracing']

In [ ]:
gov_selected_non_eco_measures = gov_oxford_select[non_economical_measures]

In [ ]:
gov_selected_non_eco_measures.head()

In [ ]:
gov_selected_non_eco_measures[['ConfirmedCases']].isnull().values.any()

In [ ]:
gov_selected_non_eco_measures[gov_selected_non_eco_measures.isna().any(axis=1)]

In [ ]:
gov_selected_non_eco_measures[(gov_selected_non_eco_measures['CountryName'] == 'Germany') & (gov_selected_non_eco_measures['S6_Restrictions on internal movement'] == 2)]

We thus can delete `nan` rows 

In [ ]:
gov_selected_non_eco_measures = gov_selected_non_eco_measures.dropna()

In [ ]:
gov_selected_non_eco_measures[(gov_selected_non_eco_measures['CountryName'] == 'Germany') & (gov_selected_non_eco_measures['S6_Restrictions on internal movement'] == 2)]

In [ ]:
binary_format = {1:0}
binary_format_contact_tracing = {1: 2}

In [ ]:
gov_selected_non_eco_measures[['S1_School closing', 'S2_Workplace closing', 'S3_Cancel public events', 'S4_Close public transport', 'S6_Restrictions on internal movement']] = gov_selected_non_eco_measures[['S1_School closing', 'S2_Workplace closing', 'S3_Cancel public events', 'S4_Close public transport', 'S6_Restrictions on internal movement']].replace(binary_format)
gov_selected_non_eco_measures[['S13_Contact tracing']] = gov_selected_non_eco_measures[['S13_Contact tracing']].replace(binary_format_contact_tracing)

In [ ]:
def locate_changes(df, column_to_change):
    if df['value'] == False:
        df['value'] = df[column_to_change]
    else:
        df['value'] = np.nan
    return df

def delete_first_row(df):
    return df[1:]



School measure

In [ ]:
school = gov_selected_non_eco_measures[['CountryName', 'Date', 'S1_School closing']]

In [ ]:
school['value'] = school['S1_School closing'].eq(school['S1_School closing'].shift())

In [ ]:
school = school.apply(lambda x: locate_changes(x, 'S1_School closing'), axis = 1)

In [ ]:
school = school.drop(columns = ['S1_School closing']).dropna()

In [ ]:
school_text_measures = {0: 'Schools open', 2: 'School closed '}

In [ ]:
school = school.replace(school_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

In [ ]:
school

Workplace measures

In [ ]:
workplace = gov_selected_non_eco_measures[['CountryName', 'Date', 'S2_Workplace closing']]
workplace['value'] = workplace['S2_Workplace closing'].eq(workplace['S2_Workplace closing'].shift())
workplace = workplace.apply(lambda x: locate_changes(x, 'S2_Workplace closing'), axis = 1)
workplace = workplace.drop(columns = ['S2_Workplace closing']).dropna()
workplace_text_measures = {0: 'Workplace open', 2: 'Worplace closed '}
workplace = workplace.replace(workplace_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

Public events measures

In [ ]:
events = gov_selected_non_eco_measures[['CountryName', 'Date', 'S3_Cancel public events']]
events['value'] = events['S3_Cancel public events'].eq(events['S3_Cancel public events'].shift())
events = events.apply(lambda x: locate_changes(x, 'S3_Cancel public events'), axis = 1)
events = events.drop(columns = ['S3_Cancel public events']).dropna()
events_text_measures = {0: 'No measures on public events', 2: 'Cancel public events'}
events = events.replace(events_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

Public transport measures

In [ ]:
transport = gov_selected_non_eco_measures[['CountryName', 'Date', 'S4_Close public transport']]
transport['value'] = transport['S4_Close public transport'].eq(transport['S4_Close public transport'].shift())
transport = transport.apply(lambda x: locate_changes(x, 'S4_Close public transport'), axis = 1)
transport = transport.drop(columns = ['S4_Close public transport']).dropna()
transport_text_measures = {0: 'No measures on public transport', 2: 'Close public transport'}
transport = transport.replace(transport_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

Information campaign measures

In [ ]:
campaign = gov_selected_non_eco_measures[['CountryName', 'Date', 'S5_Public information campaigns']]
campaign['value'] = campaign['S5_Public information campaigns'].eq(campaign['S5_Public information campaigns'].shift())
campaign = campaign.apply(lambda x: locate_changes(x, 'S5_Public information campaigns'), axis = 1)
campaign = campaign.drop(columns = ['S5_Public information campaigns']).dropna()
campaign_text_measures = {0: 'No COVID-19 information campaign', 1: 'COVID-19 public information campaign'}
campaign = campaign.replace(campaign_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

Internal movement measures

In [ ]:
internal_movement = gov_selected_non_eco_measures[['CountryName', 'Date', 'S6_Restrictions on internal movement']]
internal_movement['value'] = internal_movement['S6_Restrictions on internal movement'].eq(internal_movement['S6_Restrictions on internal movement'].shift())
internal_movement = internal_movement.apply(lambda x: locate_changes(x, 'S6_Restrictions on internal movement'), axis = 1)
internal_movement = internal_movement.drop(columns = ['S6_Restrictions on internal movement']).dropna()
internal_movement_text_measures = {0: 'No restriction on internal movement', 2: 'Rectriction on movement'}
internal_movement = internal_movement.replace(internal_movement_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

Contact tracing measures

In [ ]:
tracing = gov_selected_non_eco_measures[['CountryName', 'Date', 'S13_Contact tracing']]
tracing['value'] = tracing['S13_Contact tracing'].eq(tracing['S13_Contact tracing'].shift())
tracing = tracing.apply(lambda x: locate_changes(x, 'S13_Contact tracing'), axis = 1)
tracing = tracing.drop(columns = ['S13_Contact tracing']).dropna()
tracing_text_measures = {0: 'No contact tracing', 2: 'Contact tracing used'}
tracing = tracing.replace(tracing_text_measures).groupby('CountryName').apply(lambda x: delete_first_row(x))

Concatenate all the values

In [ ]:
overall_measures = school.append(workplace, ignore_index=True).append(events, ignore_index=True).append(transport, ignore_index=True).append(campaign, ignore_index=True).append(internal_movement, ignore_index=True).append(tracing, ignore_index=True)
overall_measures.head()

In [ ]:
germany_measures = overall_measures[overall_measures['CountryName'] == 'Germany']
usa_measures = overall_measures[overall_measures['CountryName'] == 'France']
france_measures = overall_measures[overall_measures['CountryName'] == 'United States']
gelgium_measures = overall_measures[overall_measures['CountryName'] == 'Belgium']

# Detailed Datasets

### Structure:
- a) Clean data for general informations by date
- b) Clean data for age by date
- c) Clean data for gender by date
- d) Clean data for hospitalisations by date
- e) Compute fatality rate by date

## 1 France

(Only hospitalisation data)

In [ ]:
PATH_COVID_FR = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/donnees-hospitalieres-covid19-2020-04-25-19h00.csv')
PATH_COVID_FR_METADATA = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/metadonnees-donnees-hospitalieres-covid19.csv')

PATH_COVID_FR_AGE = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/donnees-hospitalieres-classe-age-covid19-2020-04-25-19h00.csv')
PATH_COVID_FR_AGE_METADATA = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/metadonnees-donnees-hospitalieres-covid19-classes-age.csv')
PATH_COVID_FR_SEXE_METADATA = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/metadonnees-sexe.csv')

PATH_COVID_FR_HOSP = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/donnees-hospitalieres-nouveaux-covid19-2020-04-25-19h00.csv')
PATH_COVID_FR_HOSP_METADATA = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/metadonnees-hospit-incid.csv')


In [ ]:
covid_fr = pd.read_csv(PATH_COVID_FR, sep=';')
covid_fr_metadata = pd.read_csv(PATH_COVID_FR_METADATA, sep=';')

covid_fr_age = pd.read_csv(PATH_COVID_FR_AGE, sep=';')
covid_fr_age_metadata = pd.read_csv(PATH_COVID_FR_AGE_METADATA, sep=';')
covid_fr_sexe_metadata = pd.read_csv(PATH_COVID_FR_SEXE_METADATA)

covid_fr_hosp = pd.read_csv(PATH_COVID_FR_HOSP, sep=';')
covid_fr_hosp_metadata = pd.read_csv(PATH_COVID_FR_HOSP_METADATA, sep=';')

In [ ]:
covid_fr.head()

In [ ]:
covid_fr_metadata

In [ ]:
covid_fr_age.head()

In [ ]:
covid_fr_age_metadata.head(7)

In [ ]:
covid_fr_sexe_metadata

In [ ]:
covid_fr_hosp.head()

In [ ]:
covid_fr_hosp_metadata.head(6)

### A) 

ok with general dataset

### B) Clean data for age by date

In [ ]:
covid_fr_age_regions = covid_fr_age.copy()
covid_fr_age = covid_fr_age.rename({'cl_age90':'age', 'jour':'date', 'hosp':'hospitalisations', 'rea':'reanimations', 'dc':'deaths'}, axis=1).drop(['rad', 'reg'], axis=1)

In [ ]:
covid_fr_age = covid_fr_age.groupby(['age', 'date']).sum().reset_index()

In [ ]:
covid_fr_age.head()

### C) Clean data for sexe by date

In [ ]:
covid_fr_sexe_regions = covid_fr.copy()
covid_fr_sexe = covid_fr.rename({'jour':'date', 'hosp':'hospitalisations', 'rea':'reanimations', 'dc':'deaths'}, axis=1).drop(['dep', 'rad'], axis=1)

In [ ]:
covid_fr_sexe['sexe'] = covid_fr_sexe['sexe'].map({0:'total', 1:'m', 2:'f'})

In [ ]:
covid_fr_sexe = covid_fr_sexe.groupby(['sexe', 'date']).sum().reset_index()

In [ ]:
covid_fr_sexe.head()

## 2 Germany

In [ ]:
PATH_COVID_GER = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/RKI_COVID19.csv')

In [ ]:
covid_ger = pd.read_csv(PATH_COVID_GER)

In [ ]:
covid_ger = covid_ger.rename({'Altersgruppe':'Age', 'Geschlecht':'Sexe', 'AnzahlFall':'Cases', 'AnzahlTodesfall':'Deaths', 'Meldedatum':'Date', 'NeuerFall':'New Cases', 'NeuerTodesfall':'New Deaths', 'NeuGenesen':'New Recovers', 'AnzahlGenesen':'Recovers'}, axis=1)

In [ ]:
covid_ger.head()

### a) Clean data for general informations by date

In [ ]:
covid_ger_general = covid_ger[['Bundesland', 'Landkreis', 'Cases', 'Deaths', 'Date', 'Recovers']]

In [ ]:
covid_ger_general_by_regions = covid_ger_general.groupby(['Bundesland', 'Landkreis', 'Date']).sum().reset_index()

In [ ]:
covid_ger_general_by_regions.head()

In [ ]:
covid_ger_general_by_date = covid_ger_general_by_regions.drop(['Bundesland', 'Landkreis'], axis=1)\
                                    .groupby('Date')\
                                    .sum()\
                                    .reset_index()

In [ ]:
covid_ger_general_by_date.head()

### b) Clean data for age by date

In [ ]:
covid_ger_general_age = covid_ger[['Bundesland', 'Landkreis', 'Cases', 'Deaths', 'Date', 'Recovers', 'Age']]

In [ ]:
covid_ger_general_age_by_region = covid_ger_general_age.groupby(['Bundesland', 'Landkreis', 'Date', 'Age'])\
                                        .sum()\
                                        .reset_index()

In [ ]:
covid_ger_general_age_by_region.head()

In [ ]:
covid_ger_general_age_by_region_by_date = covid_ger_general_age.drop(['Bundesland', 'Landkreis'], axis=1)\
                                                .groupby(['Date', 'Age'])\
                                                .sum()\
                                                .reset_index()

In [ ]:
covid_ger_general_age_by_region_by_date.head()

### c) Clean data for sexe by date

In [ ]:
covid_ger_general_sexe = covid_ger[['Bundesland', 'Landkreis', 'Cases', 'Deaths', 'Date', 'Recovers', 'Sexe']]

In [ ]:
covid_ger_general_sexe_by_region = covid_ger_general_sexe.groupby(['Bundesland', 'Landkreis', 'Date', 'Sexe'])\
                                        .sum()\
                                        .reset_index()

In [ ]:
covid_ger_general_sexe_by_region.head()

In [ ]:
covid_ger_general_sexe_by_region_by_date = covid_ger_general_sexe.drop(['Bundesland', 'Landkreis'], axis=1)\
                                                .groupby(['Date', 'Sexe'])\
                                                .sum()\
                                                .reset_index()

In [ ]:
covid_ger_general_sexe_by_region_by_date.head()

### e) Compute fatality rate by date

In [ ]:
def compute_fatality_rate(df):
    df['fatality_rate'] = df['Deaths'] / df['Cases']
    return df

In [ ]:
covid_ger_fatality_rate_by_region = covid_ger[['Bundesland', 'Landkreis', 'Cases', 'Deaths', 'Date']]\
                                        .groupby(['Bundesland', 'Landkreis', 'Date'])\
                                        .sum()\
                                        .reset_index()

In [ ]:
covid_ger_fatality_rate_by_date = covid_ger[['Cases', 'Deaths', 'Date']]\
                                        .groupby('Date')\
                                        .sum()\
                                        .reset_index()

In [ ]:
compute_fatality_rate(covid_ger_fatality_rate_by_region).head()

In [ ]:
compute_fatality_rate(covid_ger_fatality_rate_by_date).head()

#### Comments

After doing e), we should only run the function compute_fatality_rate on the df computed in b) and c).

## 3 Belgium

In [ ]:
PATH_COVID_BE = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/COVID19BE.xlsx')
PATH_COVID_BE_HOSP = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/COVID19BE_HOSP.csv')
PATH_COVID_BE_DEATHS = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/COVID19BE_MORT.csv')
PATH_COVID_BE_TESTS = os.path.expanduser('~/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/COVID19BE_tests.csv')


In [ ]:
covid_be = pd.read_excel(PATH_COVID_BE)
covid_be_hops = pd.read_csv(PATH_COVID_BE_HOSP, encoding = "ISO-8859-1")
covid_be_deaths = pd.read_csv(PATH_COVID_BE_DEATHS)
covid_be_tests = pd.read_csv(PATH_COVID_BE_TESTS)

In [ ]:
covid_be.head()

In [ ]:
covid_be_hops.head()

In [ ]:
covid_be_deaths.head()

In [ ]:
covid_be_tests.head()

### b) Clean data for age by date

In [ ]:
covid_be['AGEGROUP'].unique()

In [ ]:
covid_be_deaths['AGEGROUP'].unique()

In [ ]:
covid_be_cases_age_by_region = covid_be.drop('SEX', axis=1)\
                                    .groupby(['DATE', 'PROVINCE', 'REGION', 'AGEGROUP'])\
                                    .sum()\
                                    .reset_index()

In [ ]:
covid_be_cases_age_by_region.head()

In [ ]:
covid_be_cases_age_by_date = covid_be.drop(['SEX', 'PROVINCE', 'REGION'], axis=1)\
                                    .groupby(['DATE', 'AGEGROUP'])\
                                    .sum()\
                                    .reset_index()

In [ ]:
covid_be_cases_age_by_date.head()

In [ ]:
covid_be_deaths_age_by_region = covid_be_deaths.drop('SEX', axis=1)\
                                    .groupby(['DATE', 'REGION', 'AGEGROUP'])\
                                    .sum()\
                                    .reset_index()

In [ ]:
covid_be_deaths_age_by_region.head()

In [ ]:
covid_be_deaths_age_by_date = covid_be_deaths.drop(['SEX', 'REGION'], axis=1)\
                                    .groupby(['DATE', 'AGEGROUP'])\
                                    .sum()\
                                    .reset_index()

In [ ]:
covid_be_deaths_age_by_date.head()

### c) Clean data for sexe by date

In [ ]:
covid_be_cases_sexe_by_region = covid_be.drop('AGEGROUP', axis=1)\
                                    .groupby(['DATE', 'PROVINCE', 'REGION', 'SEX'])\
                                    .sum()\
                                    .reset_index()

In [ ]:
covid_be_cases_sexe_by_region.head()

In [ ]:
covid_be_cases_sexe_by_date = covid_be.drop(['PROVINCE', 'AGEGROUP', 'REGION'], axis=1)\
                                .groupby(['DATE', 'SEX']).sum().reset_index()\
                                .merge(covid_be_deaths.drop(['AGEGROUP', 'REGION'], axis=1).groupby(['DATE', 'SEX']).sum().reset_index(),
                                      left_on=['SEX', 'DATE'],
                                      right_on=['SEX', 'DATE'])

In [ ]:
covid_be_cases_sexe_by_date['fatality_rate'] = covid_be_cases_sexe_by_date['DEATHS'] / covid_be_cases_sexe_by_date['CASES']

In [ ]:
covid_be_cases_sexe_by_date.head()

## 4 New York

In [ ]:
PATH_COVID_NY = os.path.expanduser('/Users/olivier/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/newyork/case-hosp-death.csv')

PATH_COVID_NY_AGE = os.path.expanduser('/Users/olivier/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/newyork/by-age.csv')
PATH_COVID_NY_SEXE = os.path.expanduser('/Users/olivier/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/newyork/by-sex.csv')

PATH_COVID_NY_TEST = os.path.expanduser('/Users/olivier/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/newyork/tests-by-zcta.csv')

In [ ]:
covid_ny = pd.read_csv(PATH_COVID_NY)

covid_ny_age = pd.read_csv(PATH_COVID_NY_AGE)
covid_ny_sexe = pd.read_csv(PATH_COVID_NY_SEXE)

covid_ny_test = pd.read_csv(PATH_COVID_NY_TEST)

In [ ]:
covid_ny.head()

In [ ]:
covid_ny_age

In [ ]:
covid_ny_sexe

In [ ]:
covid_ny_test.head()

## 5 California

In [ ]:
PATH_COVID_CLFN = os.path.expanduser('/Users/olivier/Documents/GitHub/com-480-project-pouletpanier/data/detailed_data/california-coronavirus-data/cdph-state-totals.csv')

In [ ]:
covid_clfn = pd.read_csv(PATH_COVID_CLFN)

In [ ]:
covid_clfn.head()

In [ ]:
covid_clfn.columns

### b) Clean data for age by date

In [ ]:
covid_clfn_age = covid_clfn[['date', 'confirmed_cases', 'deaths', 'age_0_to_17', 'age_18_to_49', 'age_50_to_64',
       'age_65_and_up', 'age_unknown']]

In [ ]:
covid_clfn_age.head()

### c) Clean data for sexe by date

In [ ]:
covid_clfn_sexe = covid_clfn[['date', 'confirmed_cases', 'deaths', 'gender_male',
       'gender_female', 'gender_unknown']]

In [ ]:
covid_clfn_sexe.head()

### d) Clean data for hospitalisations by date


In [ ]:
covid_clfn_hosp = covid_clfn[['date','confirmed_cases', 'deaths', 'confirmed_hospitalizations', 'confirmed_icu',
       'suspected_hospitalizations', 'suspected_icu',
       'healthcare_worker_infections', 'healthcare_worker_deaths']]

In [ ]:
covid_clfn_hosp.head()